In [ ]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

# === Step 1: Load datasets ===
df_songs = pd.read_csv("ex.csv")
df_interact = pd.read_csv("Processed_User_Song_Interaction.csv")

In [ ]:
# === Step 2: Process metadata ===
metadata = df_songs[['Genre', 'Album/Movie']].fillna('Unknown')
encoder = OneHotEncoder()
meta_encoded = encoder.fit_transform(metadata).toarray()

df_songs['User-Rating'] = df_songs['User-Rating'].str.replace('/10', '').astype(float)
ratings = df_songs[['User-Rating']].fillna(df_songs['User-Rating'].mean())

meta_features = np.hstack([meta_encoded, ratings])

# PCA on song metadata
pca = PCA(n_components=20, random_state=42)
meta_pca = pca.fit_transform(meta_features)

In [ ]:
# === Step 3: Train SVD model ===
reader = Reader(rating_scale=(0, df_interact['listen_count'].max()))
data = Dataset.load_from_df(df_interact[['user_id', 'song_index', 'listen_count']], reader)
trainset = data.build_full_trainset()

model = SVD(n_factors=50, random_state=42)
model.fit(trainset)

svd_vectors = model.qi  # item (song) vectors

In [ ]:
# === Step 4: Combine SVD + PCA features ===
combined_vectors = np.hstack([svd_vectors, meta_pca])

In [ ]:
# === Step 5: Recommend for user ===
def recommend_for_user(user_id, top_k=10):
    user_vector = model.pu[trainset.to_inner_uid(user_id)]
    similarities = cosine_similarity(
        [np.hstack([user_vector, np.zeros(meta_pca.shape[1])])],
        combined_vectors
    ).flatten()

    listened = df_interact[df_interact.user_id == user_id]['song_index'].tolist()
    unlistened = [i for i in range(len(df_songs)) if i not in listened]

    ranked = sorted(unlistened, key=lambda x: -similarities[x])
    top_indices = ranked[:top_k]
    return df_songs.iloc[top_indices][['Song-Name', 'Singer/Artists', 'Genre']]

In [ ]:
# === Step 6: Try recommendation ===
recommend_for_user("user_52")